In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import os.path as op
import shutil
from zipfile import ZipFile
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from utils import get_image_paths, word_from_image_path, preprocess_image, print_im, TextTransform, N_CHARS, SEQUENCE_LENGTH, IMAGE_DIMENSIONS
from tqdm import tqdm, tqdm_notebook

Using TensorFlow backend.


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from multi_gpu import make_parallel

In [4]:
base_dir = '/mnt/mjsynth/mnt/ramdisk/max/90kDICT32px/'

In [27]:
%%time
images_paths = get_image_paths(base_dir)

CPU times: user 19.9 s, sys: 3.18 s, total: 23 s
Wall time: 22.3 s


In [28]:
len(images_paths)

8919273

In [29]:
text_transformer = TextTransform()

### creating X and Y

In [8]:
#%%time
ims = []
errors_1, errors_2 = [], []
for i, im in tqdm_notebook(enumerate(images_paths[:]), total=len(images_paths)):
    try:
        ims.append(preprocess_image(img_to_array(load_img(im, grayscale=True))))
    except OSError:
        errors_1.append(i)

KeyboardInterrupt: 

### save on disk

In [20]:
batch1 = np.array(ims[:1000000])
ims = ims[1000000:]
batch2 = np.array(ims[:1000000])
one_million = 1000000
del ims[:one_million]
batch3 = np.array(ims[:one_million])
del ims[:one_million]
batch4 = np.array(ims[:one_million])
del ims[:one_million]
batch5 = np.array(ims[:one_million])
del ims[:one_million]
batch6 = np.array(ims[:one_million])
del ims[:one_million]
batch7 = np.array(ims[:one_million])
del ims[:one_million]
batch8 = np.array(ims[:one_million])
del ims[:one_million]
batch9 = np.array(ims)

In [32]:
import h5py

with h5py.File('/mnt/x1.py', 'w') as hf:
    hf.create_dataset("x",  data=batch1)
with h5py.File('/mnt/x2.py', 'w') as hf:
    hf.create_dataset("x",  data=batch2)
with h5py.File('/mnt/x3.py', 'w') as hf:
    hf.create_dataset("x",  data=batch3)
with h5py.File('/mnt/x4.py', 'w') as hf:
    hf.create_dataset("x",  data=batch4)
with h5py.File('/mnt/x5.h5', 'w') as hf:
    hf.create_dataset("x",  data=batch5)
with h5py.File('/mnt/x6.h5', 'w') as hf:
    hf.create_dataset("x",  data=batch6)
with h5py.File('/mnt/x7.h5', 'w') as hf:
    hf.create_dataset("x",  data=batch7)
with h5py.File('/mnt/x8.h5', 'w') as hf:
    hf.create_dataset("x",  data=batch8)
with h5py.File('/mnt/x9.h5', 'w') as hf:
    hf.create_dataset("x",  data=batch9)
with h5py.File('/mnt/errors.h5', 'w') as hf:
    hf.create_dataset("x",  data=np.array(errors_1))

In [58]:
del batch1, batch2, batch3, batch4, batch5, batch6, batch7, batch8, batch9

In [ ]:
del ims

In [31]:
image_paths_without_errors = []

for i, im in tqdm_notebook(enumerate(images_paths), total=len(images_paths)):
    if i not in errors_1:
        image_paths_without_errors.append(im)

In [32]:
word_from_image_path(image_paths_without_errors[0])

'Buckram'

In [33]:
print("OK")

OK


In [34]:
%%time
batch_y = text_transformer.make_batch_labels(image_paths_without_errors)

CPU times: user 7min 37s, sys: 5min 38s, total: 13min 15s
Wall time: 20min 31s


In [35]:
b_y = batch_y.astype(np.float32)

In [38]:
del batch_y

In [42]:
with h5py.File('/datadrive/y.h5', 'w') as hf:
    hf.create_dataset("x",  data=np.array(b_y))

In [40]:
import pickle

In [41]:
with open('/datadrive/tt_new.pickle', 'wb+') as f:
    pickle.dump(text_transformer, f)

### Read data

In [9]:
import h5py
import numpy as np

In [30]:
with h5py.File('/mnt/errors.h5', 'r') as hf:
    errors_1 = hf['x'][:]

In [8]:
%%time
with h5py.File('/mnt/x_5m_minus_min_div_std.h5', 'r') as hf:
    x1 = hf['x'][:]

In [5]:
with h5py.File('/mnt/x1.h5', 'r') as hf:
    x1 = hf['x'][:]

In [6]:
for i in range(2, 6):
    print(i)
    with h5py.File('/mnt/x{}.h5'.format(i), 'r') as hf:
        x1 = np.concatenate((x1, hf['x'][:]))

2
3
4
5


In [10]:
mean = x1.mean(axis=(1, 2))

In [24]:
%%time
std = x1.std(axis=(1, 2))

CPU times: user 35.5 s, sys: 1min 47s, total: 2min 23s
Wall time: 2min 23s


### dump std, mean

In [19]:
with h5py.File('/mnt/mean.h5', 'w') as hf:
    hf.create_dataset('mean', data=mean)

with h5py.File('/mnt/std.h5', 'w') as hf:
    hf.create_dataset('std', data=std)

In [25]:
%%time
x1 = (x1 - mean.reshape((-1, 1, 1))) / std.reshape((-1, 1, 1))

CPU times: user 24.9 s, sys: 1min 51s, total: 2min 16s
Wall time: 2min 17s


### save x and y

In [18]:
import h5py

In [ ]:
with h5py.File('/mnt/y.h5', 'w') as hf:
    hf.create_dataset("y",  data=batch_y)

In [26]:
with h5py.File('/mnt/x_5M_minus_min_div_std.h5', 'w') as hf:
    hf.create_dataset("x",  data=x1)

### load final x and y

In [5]:
import h5py
import pickle

In [6]:
%%time
with h5py.File('/datadrive/y.h5', 'r') as hf:
    y = hf['x'][:5000000]

CPU times: user 0 ns, sys: 6.37 s, total: 6.37 s
Wall time: 6.34 s


In [ ]:
%%time
with h5py.File('/datadrive/x_5m_minus_min_div_std.h5', 'r') as hf:
    x = hf['x'][:]

In [ ]:
with open('tt.pickle', 'rb') as f:
    tt = pickle.load(f)

### Model creation

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Reshape, Activation
from keras.layers import Flatten

In [ ]:
from utils import IMAGE_DIMENSIONS, SEQUENCE_LENGTH, N_CHARS

In [ ]:
convolutions = [64, 128, 256, 512, 512]
kernels = [5, 5, 3, 3, 3]

model = Sequential()
input_shape = (None,) + IMAGE_DIMENSIONS + (1,)

model.add(Conv2D(nb_filter=64,
                     nb_row=kernels[0],
                     nb_col=kernels[0],
                     activation='relu',
                     border_mode='same',
                     batch_input_shape=input_shape, name="convo" + str(0)))

model.add(MaxPooling2D(pool_size=(2, 2), border_mode='same'))

for i, (kernel, convolution_size) in enumerate(zip(convolutions[1:], kernels[1:])):
    model.add(Conv2D(nb_filter=convolution_size,
                     nb_row=kernel,
                     nb_col=kernel,
                     activation='relu',
                     border_mode='same',
                     name="convo" + str(i + 1)))
    
    if i <= 3 :
        model.add(MaxPooling2D(pool_size=(2, 2), border_mode='same',))

model.add(Flatten())
# model.add(Dense(128, activation='relu'))
model.add(Dense(4096, activation='relu'))
model.add(Dense(4096, activation='relu'))

model.add(Dense(SEQUENCE_LENGTH * N_CHARS))

model.add(Reshape((SEQUENCE_LENGTH, N_CHARS)))
model.add(Activation('softmax'))


model.compile(optimizer='adam', loss='categorical_crossentropy')

In [ ]:
p_model = make_parallel(model, 4)
p_model.compile(optimizer='adam', loss='categorical_crossentropy')

In [32]:
# model.save('model_random_weights.keras')

In [ ]:
tt.word_from_matrix(y[0])

In [ ]:
one_million = 1000000
hundred_thousands = 100000

In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
import datetime

In [22]:
for i in range(251):
    begin = ((i - 1) % 50) * hundred_thousands
    end = (((i - 1) % 50) + 1) * hundred_thousands
    run = i // 50 + 1
    
    x_batch = x[begin:end]
    y_batch = y[begin:end]
    
    history = p_model.fit(x_batch.reshape((x_batch.shape[0],) + IMAGE_DIMENSIONS + (1,)), y_batch, nb_epoch=1)
    
    now = datetime.datetime.now().isoformat().split('.')[0]

    with open("/datadrive/state", 'a+') as f:
        f.write("run={} epoch={} loss={} now={} from={} to={}\n".format(run, (i - 1) % 50, history.history['loss'][0], now, begin, end))
    
    if i and not i % 10:
        p_model.save('/datadrive/p_model_{}.h5'.format(i))

Epoch 1/1
   544/100000 [..............................] - ETA: 85539s - loss: 2.5774 

KeyboardInterrupt: 